In [2]:
import pandas as pd
data = pd.read_csv("sigma.csv")
# null
data = data.drop_duplicates()
data.duplicated().sum()



0

In [3]:
data

,label,text
0,Ismail Han,😺
1,Ismail Han,Kaz ta hoilo
2,Ismail Han,Kotha na boijai
3,Ismail Han,Ami vavlam tmi ethan
4,Ismail Han,What's up?
...,...,...
106391,Tasin Mahatab,Ata sikis
106392,Tasin Mahatab,V .V.T question
106393,Tasin Mahatab,Most of the important of reproduction😆Ismail Han
106398,Tasin Mahatab,Hu


#### 1st e word tokenize
##### puncuation remove
#### remove special char
#### remove stop word
#### Lemmatization
#### Label Encoder

In [4]:
#from nltk.tokenize import word_tokenize
#import nltk
import banglanltk as nltk
from banglanltk import word_tokenize
#nltk.download('wordnet')
#nltk.download('omw-1.4')
#nltk.download('punkkt_tab')

data['text'] = data['text'].fillna('') # nan(not a number) also kind of NONE type thing. ekhane amra oi value gula change kore dibo
data['text'].apply(lambda x: word_tokenize(x))


0                                                       [😺]
1                                          [Kaz, ta, hoilo]
2                                       [Kotha, na, boijai]
3                                 [Ami, vavlam, tmi, ethan]
4                                               [Whats, up]
                                ...                        
106391                                         [Ata, sikis]
106392                                    [V, VT, question]
106393    [Most, of, the, important, of, reproduction😆Is...
106398                                                 [Hu]
106399                                     [Eto, grp, lage]
Name: text, Length: 70513, dtype: object

In [5]:
# remove punc
import string
def remove_punc(text):
  trans = str.maketrans('', '', string.punctuation)
  return text.translate(trans)

data['text'] = data['text'].apply(remove_punc)
data['text']

0                                                        😺
1                                             Kaz ta hoilo
2                                          Kotha na boijai
3                                     Ami vavlam tmi ethan
4                                                 Whats up
                                ...                       
106391                                           Ata sikis
106392                                       V VT question
106393    Most of the important of reproduction😆Ismail Han
106398                                                  Hu
106399                                        Eto grp lage
Name: text, Length: 70513, dtype: object

In [6]:
### remove special char (abcd er baire habijabi)
import re
#def remove_noise(text):
#  t = re.sub('[^a-zA-Z]', ' ', text)
#  return t
#data['text'] = data['text'].apply(remove_noise)
#data['text']

In [7]:
### remove stop word
import nltk
from bn_nlp.preprocessing import ban_processing
nltk.download('stopwords')
from bn_nlp.preprocessing import ban_processing
bp=ban_processing()
from nltk.corpus import stopwords
# bangla
with open('bn_nlp/dataset/stop_word.txt','r') as sto:
    #stops=sto.read()
    stops=set(sto.read().splitlines())
    
sw = set(stopwords.words('english'))
combined_stopwords = sw.union(stops)
def remove_sws(text):
    # low char
    try:
        text = text.lower()
    except Exception as e:
        pass
    non_stop_word = []
    split_text = text.split()
    for new in split_text:
        if new not in combined_stopwords:
            non_stop_word.append(new)
            # from nltk.stem import WordNetLemmatizer
            # lemmatizer = WordNetLemmatizer()
            # new = lemmatizer.lemmatize(new)
            # non_stop_word.append(new)
    return " ".join(non_stop_word)
            
# def remove_sws(text):
#   s = [word.lower() for word in text.split() if word.lower() not in sw]
#   return " ".join(s)
data['text'] = data['text'].apply(remove_sws)
data['text']

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0                                         😺
1                              kaz ta hoilo
2                           kotha na boijai
3                      ami vavlam tmi ethan
4                                     whats
                        ...                
106391                            ata sikis
106392                        v vt question
106393    important reproduction😆ismail han
106398                                   hu
106399                         eto grp lage
Name: text, Length: 70513, dtype: object

In [8]:
### Lemmatization
# similar word
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
def lemma(text):
    
    lemmatize_Word = []
    split_text = text.split()
    for new in split_text:
        if new not in sw:
            lemmatize_Word.append(new)
            from nltk.stem import WordNetLemmatizer
            lemmatizer = WordNetLemmatizer()
            new = lemmatizer.lemmatize(new)
            lemmatize_Word.append(new)
    return " ".join(lemmatize_Word)
           
def lemma(text):
  l = [lemmatizer.lemmatize(word) for word in text.split()]
  return " ".join(l)

data['text'] = data['text'].apply(lemma)
data['text']

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


0                                         😺
1                              kaz ta hoilo
2                           kotha na boijai
3                      ami vavlam tmi ethan
4                                     whats
                        ...                
106391                            ata sikis
106392                        v vt question
106393    important reproduction😆ismail han
106398                                   hu
106399                         eto grp lage
Name: text, Length: 70513, dtype: object

In [9]:
### Label Encoder
# from sklearn.preprocessing import LabelEncoder
# encoder =LabelEncoder()

# data['v1']=encoder.fit_transform(data['v1'])
# data

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_features=3000)
x = tf.fit_transform(data['text']).toarray()
y = data['label']
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(x)
from sklearn.naive_bayes import  MultinomialNB
# Create the instance of Naive Bayes
clf = MultinomialNB()
# Fit the data
clf.fit(X_train, y_train)
# Making prediction



y_pred = clf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Classification Report:\n", classification_report(y_test, y_pred))



# your_sen = ["join the meeting"]
# your_sen = tf.transform(your_sen).toarray()
# y_pred = clf.predict(X_test)
# print(y_pred)
# print("Accuracy Score: ", accuracy_score(y_test, y_pred))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Accuracy: 47.32%


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification Report:
                                                   precision    recall  f1-score   support

                                         AH Emon       0.00      0.00      0.00         1
                                 Aarvi Chowdhury       0.00      0.00      0.00         2
                            Abderrahim Es-sabbar       0.00      0.00      0.00         1
                               Abdoullahi Øllaid       0.00      0.00      0.00         1
                            Abdullah Tousif Nafi       0.00      0.00      0.00         3
                                     Abrar Siyam       0.00      0.00      0.00         1
                                      Abu Rayhan       0.00      0.00      0.00         3
                                       Abu Sayem       0.00      0.00      0.00         2
                              Abu Shahed Ibrahim       0.00      0.00      0.00         1
                             Adam James Clements       0.00      0.00      

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [29]:
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    text = text.lower()  # Lowercase
    text = ''.join([char for char in text if char not in string.punctuation])  # Remove punctuation
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

# tokeize korte hobe
your_sen = ["msg dibo na"]

# Preprocess the new sentence before vectorizing
your_sen = [preprocess_text(sen) for sen in your_sen]

# Transform the new sentence using the same vectorizer
your_sen_transformed = tf.transform(your_sen).toarray()
#data['v2'].apply(lambda x: word_tokenize(x))
# Predict the label for the new sentence
your_sen_pred = clf.predict(your_sen_transformed)

# Print the prediction for the new sentence
print("Prediction for your sentence:")
print(your_sen_pred[0],"chuppppp")







Prediction for your sentence:
Ismail Han chuppppp


In [12]:
print("hi")

hi
